# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [59]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [60]:
city_data = pd.read_csv('clean_city_data.csv')
city_data.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Sumbe,-11.21,13.84,76.28,83,100,8.86,AO,1603481567
1,Vaini,-21.20,-175.20,73.40,83,75,9.17,TO,1603481530
2,Lebu,-37.62,-73.65,55.58,73,100,6.62,CL,1603481567
3,Kapaa,22.08,-159.32,79.00,88,75,8.75,US,1603481568
4,Khatanga,71.97,102.50,7.56,95,100,6.55,RU,1603481025


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [61]:
# Create locations variable
locations = city_data[["Lat", "Lng"]].astype(float)

# Configure gmaps
gmaps.configure(api_key=g_key)

# Create a Heatmap layer
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=city_data['Humidity'], 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)


In [62]:
locations['Lat']

0     -11.21
1     -21.20
2     -37.62
3      22.08
4      71.97
       ...  
565    54.77
566     3.08
567    25.38
568    14.20
569    -6.98
Name: Lat, Length: 570, dtype: float64

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [109]:
ideal_city = city_data[(city_data['Max Temp'] < 75) & (city_data['Max Temp'] > 55) & (city_data['Humidity'] < 60) & \
                         (city_data['Cloudiness'] == 0)]
ideal_city

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
112,Constitución,-35.33,-72.42,58.95,59,0,15.26,CL,1603481592
143,Mangan,27.52,88.53,67.39,26,0,5.61,IN,1603481600
153,Bulawayo,-20.15,28.58,69.35,46,0,7.00,ZW,1603481602
183,Pokhara,28.23,83.98,65.80,27,0,5.41,NP,1603481608
207,Rūdbār,30.15,62.60,57.22,35,0,11.48,AF,1603481614
233,Khuzdar,27.74,66.64,58.14,32,0,7.16,PK,1603481619
245,Punta Alta,-38.88,-62.07,70.45,55,0,13.09,AR,1603481621
269,Alice Springs,-23.70,133.88,60.80,59,0,4.70,AU,1603481627
290,Sohbatpur,28.52,68.54,72.37,18,0,7.09,PK,1603481542
312,Sabha,27.04,14.43,69.21,24,0,7.67,LY,1603481636


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [110]:
# Create hotel_df and the hotel name column
# ideal_city['Hotel Name'] = ' '
# ideal_city

In [111]:
# Set parameters to find hotels within 5000 meters
params = {'radius': 5000,
          'keyword': 'hotel',
         'type': 'lodging',
         'key': g_key}
hotel_name = []
# move through each city and locate a hotel
for index, row in ideal_city.iterrows():
    params['location'] = f"{row['Lat']}, {row['Lng']}"
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    name_address = requests.get(base_url, params=params)
    hotel_data = name_address.json()
    try:
        hotel_name.append(hotel_data['results'][0]['name'])
    except IndexError:
        hotel_name.append('No Hotel')
        pass

In [113]:
hotel_name
# add hotel_name to hotel_df
ideal_city['Hotel Name'] = hotel_name
ideal_city

C:\Users\lakew\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
112,Constitución,-35.33,-72.42,58.95,59,0,15.26,CL,1603481592,Bed & Breakfast Sweet Dreams
143,Mangan,27.52,88.53,67.39,26,0,5.61,IN,1603481600,No Hotel
153,Bulawayo,-20.15,28.58,69.35,46,0,7.00,ZW,1603481602,Banff Lodge Hotel
183,Pokhara,28.23,83.98,65.80,27,0,5.41,NP,1603481608,Shangrila Village Resort Pokhara
207,Rūdbār,30.15,62.60,57.22,35,0,11.48,AF,1603481614,No Hotel
233,Khuzdar,27.74,66.64,58.14,32,0,7.16,PK,1603481619,No Hotel
245,Punta Alta,-38.88,-62.07,70.45,55,0,13.09,AR,1603481621,Center Hotel Punta Alta
269,Alice Springs,-23.70,133.88,60.80,59,0,4.70,AU,1603481627,DoubleTree by Hilton Hotel Alice Springs
290,Sohbatpur,28.52,68.54,72.37,18,0,7.09,PK,1603481542,Kattah Hotel
312,Sabha,27.04,14.43,69.21,24,0,7.67,LY,1603481636,فندق علي


In [85]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_city.iterrows()]
locations = ideal_city[["Lat", "Lng"]]

In [86]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))